In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import time
import numpy as np
import tensorflow as tf

import os
import sys
sys.path.append(os.path.join(os.getcwd(), "models"))
sys.path.append(os.path.join(os.getcwd(), "models", "research"))

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.builders import model_builder

sys.path.append(os.path.join(os.getcwd(), "utils"))
from video_utils import *
from detection_utils import *
from moi_utils import *
from submission_utils import *
from sort import *

import warnings
warnings.filterwarnings('ignore')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Preparing object detection model:

[TensorFlow 2 Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md):

1. Right click on the Model name of the model you would like to use;
2. Click on Copy link address to copy the download link of the model;
3. Paste the link in a text editor of your choice. You should observe a link similar to download.tensorflow.org/models/object_detection/tf2/YYYYYYYY/XXXXXXXXX.tar.gz;
4. Copy the XXXXXXXXX part of the link and use it to replace the value of the MODEL_NAME variable in the code shown below;
5. Copy the YYYYYYYY part of the link and use it to replace the value of the MODEL_DATE variable in the code shown below.

In [2]:
MODEL_DATE = '20200711'
MODEL_NAME = 'efficientdet_d3_coco17_tpu-32'

print('Loading model... ', end='')
start_time = time.time()

PATH_TO_MODEL_DIR = download_model(MODEL_NAME, MODEL_DATE)
PATH_TO_CFG = PATH_TO_MODEL_DIR + "/pipeline.config"
PATH_TO_CKPT = PATH_TO_MODEL_DIR + "/checkpoint"

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-0')).expect_partial()

@tf.function
def detect_fn(image):
    """Detect objects in image."""
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model... Done! Took 44.64107346534729 seconds


# Loading COCO labels:

In [3]:
LABEL_FILENAME = 'mscoco_label_map.pbtxt'
PATH_TO_LABELS = download_labels(LABEL_FILENAME)

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Loading videos frames and ROI + MOI:

In [4]:
input_dir = './data/videos'
frame_output_dir= './data/frames' 
info_dir = './data/zones-movement_paths'

video_paths = get_videos(input_dir)
print("Discover {} videos in {}".format(len(video_paths), input_dir))

Discover 1 videos in ./data/videos


In [5]:
# Example:
# extracted_frames = extract_frames_from_video(video_paths[i], frame_output_dir)
# roi, mois = extract_video_info(video_paths[i], info_dir)

# Object detection from video frames:

In [6]:
video_output_dir = './data/videos_with_boundingbox'

start_time = time.time()
    
video_object_dectection(video_paths[0], detect_fn, category_index, 
                        frame_output_dir, info_dir, video_output_dir, 
                          output_to_video = True, 
                          from_frame = 0, to_frame = 50, time_stride = 1)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds for video {}'.format(elapsed_time, video_paths[0]))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [04:50<00:00,  5.81s/it]

Done! Took 292.87210512161255 seconds for video ./data/videos\cam_01.mp4


# Object Detection and Tracking from video frames:

In [7]:
video_output_dir = './data/videos_with_boundingbox'

start_time = time.time()

mot_tracker = Sort(max_age=3, min_hits=2, iou_threshold=0.2)

track_dict = video_object_dectection_and_tracking(video_paths[0], detect_fn, mot_tracker, category_index, 
                                              frame_output_dir, info_dir, video_output_dir, 
                                              output_to_video = True, 
                                              from_frame = 0, to_frame = 50, time_stride = 1)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds for video {}'.format(elapsed_time, video_paths[0]))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [02:51<00:00,  3.42s/it]

Done! Took 173.12773251533508 seconds for video ./data/videos\cam_01.mp4


# Determine Motion of Interest for vehicles:

In [8]:
def cosin_similarity(moi, path):
    first = path[0]
    last = path[-1]
    first_point = ((first[2] - first[0])/2, (first[3] - first[1])/2)
    last_point = ((last[2] - last[0])/2, (last[3] - last[1])/2)
    path2d = (first_point, last_point)
    a = np.array((moi[1][0] - moi[0][0], moi[1][1 ]- moi[0][1]))
    b = np.array((path2d[1][0] - path2d[0][1], path2d[1][1] - path2d[1][0]))
    return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))

In [9]:
_, mois = extract_video_info(video_paths[0], info_dir)
moi_list = counting_moi(video_paths[0], mois, track_dict, similarity_fn = cosin_similarity)

# Write the tracking results and output the submission format:

In [10]:
submission_output_dir = './data/submission'

In [11]:
write_tracking_result(video_paths[0], submission_output_dir, moi_list)

In [12]:
write_submission(submission_output_dir)